In [1]:
#IMPORT PACKAGES
from rdflib import Graph
import rdflib
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from scipy.linalg import orthogonal_procrustes
import matplotlib.pyplot as plt
import geomapi.tools.progresstools as pt

from sklearn.cluster import DBSCAN
from PIL import Image

import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
import geomapi.tools.progresstools as pt

#import utils
import context 
import utils as utl
import utils.t8_utils as t8


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [4]:
#paths
path=Path(os.getcwd()).parents[0]

print(path)
input_folder_t4=path/'data'/'t4'/'test' 
input_folder_t6=path/'data'/'t6'/'test'
class_file=path/'data'/'_classes.json'
output_folder=path/'data'/'t8'/ 'test'
os.makedirs(output_folder, exist_ok=True)

#parameters
grid_resolution = 0.01

/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024


Import Classes

In [5]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import Graph

In [6]:
graphfiles=utl.get_list_of_files(input_folder_t6,'.ttl')
for f in graphfiles[:1]: #only read the first one
    print(f'processing {ut.get_filename(f)}...')      
    wallNodes=tl.graph_path_to_nodes(f)
    for n in wallNodes:
        n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(o3d.geometry.OrientedBoundingBox.create_from_points(o3d.utility.Vector3dVector(n.orientedBounds)))

    print(f'{len(wallNodes)} wallNodes detected!')

processing 05_MedOffice_01_F2_small1_walls...
145 wallNodes detected!


Import PCD

In [7]:
pcdfiles=utl.get_list_of_files(input_folder_t4,'.laz')

for f in pcdfiles[:1]: #only read the first one
    pcdNodes=[]
    
    # check if las/pcd variable is already defined    
    print(f'processing {ut.get_filename(f)}...')      
    las = laspy.read(f) if 'las' not in globals() else las
    pcd=gmu.las_to_pcd(las,getNormals=True) if 'pcd' not in globals() else pcd # this is the slowest step
    
    #seperate initial objects
    for c in class_dict['classes']:
        if c['id'] in [255,0,1,2,3,4,5]:
            idx=np.where((las['classes']==c['id']))[0]
            class_pcd=pcd.select_by_index(idx)
            object_labels=las['objects'][idx]
            
            for j in np.unique(object_labels):
                indices=np.where(object_labels==j)[0]
                object_pcd=class_pcd.select_by_index(indices)
                pcdNodes.append(PointCloudNode(resource=object_pcd,
                                            class_id=c['id'],
                                            object_id=j,
                                            color=ut.random_color(),
                                            name=c['name']+f'_{str(j)}'))
                
            
            #all further processing will be placed here (or in functions)!
            print( c['name'], f': {len(pcdNodes)} Nodes found')    

processing 05_MedOffice_01_F2_small1...
Unassigned : 1 Nodes found
Floors : 2 Nodes found
Ceilings : 3 Nodes found
Walls : 148 Nodes found
Columns : 173 Nodes found
Doors : 289 Nodes found
Windows : 289 Nodes found


In [8]:
clutterPCDNodes=[n for n in pcdNodes if 'Unassigned' in n.subject and type(n)==PointCloudNode]
floorPCDNodes=[n for n in pcdNodes if 'Floors' in n.subject and type(n)==PointCloudNode]
ceilingPCDNodes=[n for n in pcdNodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
wallPCDNodes=[n for n in pcdNodes if 'Walls' in n.subject and type(n)==PointCloudNode]
columnPCDNodes=[n for n in pcdNodes if 'Columns' in n.subject and type(n)==PointCloudNode]
doorPCDNodes=[n for n in pcdNodes if 'Doors' in n.subject and type(n)==PointCloudNode]
windowPCDNodes=[n for n in pcdNodes if 'Windows' in n.subject and type(n)==PointCloudNode]

print(f'{len(clutterPCDNodes)} clutterNodes detected!')
print(f'{len(floorPCDNodes)} floorsNodes detected!')
print(f'{len(ceilingPCDNodes)} ceilingsNodes detected!')
print(f'{len(wallPCDNodes)} wallNodes detected!')
print(f'{len(columnPCDNodes)} columnNodes detected!')
print(f'{len(doorPCDNodes)} doorNodes detected!')
print(f'{len(windowPCDNodes)} windowNodes detected!')

1 clutterNodes detected!
1 floorsNodes detected!
1 ceilingsNodes detected!
145 wallNodes detected!
25 columnNodes detected!
116 doorNodes detected!
0 windowNodes detected!


match point clouds with graph

In [9]:
for n in clutterPCDNodes+floorPCDNodes+ceilingPCDNodes+columnPCDNodes+doorPCDNodes+windowPCDNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((las['classes']==n.class_id) & (las['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(las.xyz[idx])
    red = las['red'][idx]
    green = las['green'][idx]
    blue = las['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = las['red'][idx] >> 8 & 0xFF
        green = las['green'][idx] >> 8 & 0xFF
        blue = las['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    pcd.colors=o3d.utility.Vector3dVector(np.vstack((red,green,blue)).transpose())

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0]

In [10]:
for n in wallPCDNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((las['classes']==n.class_id) & (las['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(las.xyz[idx])
    pcd.paint_uniform_color([0.5,0.5,0.5])
    
    red = las['red'][idx]
    green = las['green'][idx]
    blue = las['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = las['red'][idx] >> 8 & 0xFF
        green = las['green'][idx] >> 8 & 0xFF
        blue = las['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    
    pcd.colors=o3d.utility.Vector3dVector(np.column_stack((red, green, blue)))

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0] 

match PointCloudNodes to BIMNodes

In [11]:
for n in wallNodes:
    n.derivedFrom = next(p for p in wallPCDNodes if p.subject.toPython() in [w.derivedFrom for w in wallNodes])

In [12]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallPCDNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

Import Reference Levels

In [13]:
# levelNodes=[n for n in nodes if 'level' in n.subject]
# referenceNodes=[]
# for l in levelNodes:
#     new_graph=ut.get_subject_graph(graph,levelNodes[0].subject)
#     n=SessionNode(graph=new_graph)
#     n.get_oriented_bounding_box()
#     n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)
#     referenceNodes.append(n) # something is wrong in the tl.graph_to_nodes function
# levelNodes=referenceNodes
# print(f'{len(levelNodes)} levelNodes detected!')

Import ceilings and floors

In [14]:
# for n in ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
#     idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
#     pcd=o3d.geometry.PointCloud()
#     pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
#     n.resource=pcd
#     n.get_oriented_bounding_box()
#     n.orientedBoundingBox.color=[1,1,0]

Load the ground truth data

In [15]:
groundtruthPath = '/home/sdegeyter/Code/Scan-to-BIM-CVPR-2024/data/t0/05_MedOffice_01_F2_doors.json'
groundtruthNodes = []

with open(groundtruthPath) as gt:
    data = json.load(gt)
    
for item in data:
    doorNode = Node(
        id = item['id'],
        width = item['width'],
        height = item['height'],
        depth = item['depth'],
        center = item['loc'], 
        rotation = item['rotation'], 
        host = item['host_id']
    )
    groundtruthNodes.append(doorNode)
print(len(groundtruthNodes))


62


## PROCESSING

Convert dtrings from the graph to arrays

In [16]:
for n in wallNodes:
    n.startpoint = np.asarray(n.startpoint[1:-1].split(), dtype=float)
    n.endpoint = np.asarray(n.endpoint[1:-1].split(), dtype=float)
    n.normal = np.asarray(n.normal[1:-1].split(), dtype=float)
    n.height = float(n.height)
    n.name = n.subject.split('///')[-1]
    

In [17]:
def points_on_line(point1, point2, step_size):
    """
    Generate points on a line between two given points with a specified step size.

    Parameters:
    - point1: The starting point of the line.
    - point2: The ending point of the line.
    - step_size: The step size between consecutive points.

    Returns:
    - points: A list of points on the line.
    """
    # Calculate the direction vector
    direction = point2 - point1

    # Calculate the length of the line segment
    length = np.linalg.norm(direction)

    # Normalize the direction vector
    direction /= length

    # Calculate the number of steps needed
    num_steps = int(length / step_size)

    # Generate points along the line
    points = np.array([point1 + i * step_size * direction for i in range(num_steps + 1)])

    return points


Load the full resolution point cloud for a more accurate result (unlabeled)

In [18]:
# laz = laspy.read(os.path.join(Path(os.getcwd()).parents[0]/'data',"full_resolution_populierenhof.las"))
full_res_point_cloud_o3d = gmu.las_to_pcd(las)

Convert the full point cloud into a mesh and add it to a raycasting scene

In [19]:

#Cut a part out of the full resolution pointcloud
# joined_pcd = full_res_point_cloud_o3d.crop(expanded_bounding_box)
#Create a messh from this point cloud 
octree=pt.pcd_to_octree(full_res_point_cloud_o3d,12) #if octree is None else octree
full_res_point_cloud_o3d = None
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh


#Create a identity array containing the color so this can be retrieved afterwards
original_colors=np.asarray(mesh.vertex_colors)
indices=np.asarray(mesh.triangles)[:,0]
triangle_colors=original_colors[indices]
#append black color at the end of the array for the invalid hits
triangle_colors=np.vstack((triangle_colors,np.array([0,0,0])))

# Create raycasting scene
scene = o3d.t.geometry.RaycastingScene()
mesh=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene.add_triangles(mesh) 

# Calculate the size of each octree node based on octree depth and overall size
def calculate_node_size(octree_depth, octree_size):
    num_voxels_per_dim = 2 ** octree_depth
    voxel_size = octree_size / num_voxels_per_dim
    return voxel_size

# Example usage:
octree_depth = octree.max_depth  # Example value for max_depth
octree_size = octree.size  # Example size of the octree in world units
voxel_size = calculate_node_size(octree_depth, octree_size)
print("Voxelsize = ", voxel_size)
octree = None

Voxelsize =  0.02052154296875


In [20]:
# def is_door(openingwidth, openingheight, t_min_width = 0.5, t_max_width = 3, t_min_height = 1.5, t_max_height = 2.3):

#     if t_min_width < openingwidth < t_max_width and t_min_height < openingheight < t_max_height:
#         return True
#     else: 
#         return False

## Method 1: Using the pointcloud and wall data to retrieve potential openings in the walls 
(Can also be used to retrieve wall detailing)

In [21]:
# def create_opening_pcds(startpoint, endpoint, height, resolution, direction, scene, offset =0.7, path = None, show = False, min_samples = 5000, eps = 0.2, wallThickness = 0.1, voxel_size = 0.01):
    
#     min_z = np.min([endpoint[2], startpoint[2]])
#     max_z = min_z + height
#     num_z_steps = int(height /resolution)
#     z_grid = np.linspace(min_z, max_z, num_z_steps)  # Adjust the number of grid points as needed
#     xyz_grid = []
#     for z in z_grid:
#         start = n.startpoint.copy()
#         end =  n.endpoint.copy()
#         start[2] = z
#         end[2] = z
#         xyz_grid.append(points_on_line(start, end, resolution))

#     grid = np.asarray(xyz_grid).reshape((-1, 3), order='C') 
    
#     openingpcds = []
#     # Create Open3D point cloud
#     grid_center_pcd = o3d.geometry.PointCloud()
#     grid_center_pcd.points = o3d.utility.Vector3dVector(np.asarray(grid))
#     grid_center_pcd.paint_uniform_color([1,0,0])

#     #In face is the dominant side of the wall
#     grid_in = grid + direction*offset
#     #out face is the other side of the dominant side
#     grid_out = grid - direction*offset

#     #create rays for the in side (towards the dominant side
#     ori_x = direction[0] * np.ones(len(grid))
#     ori_y = direction[1] * np.ones(len(grid))
#     ori_z = direction[2] * np.ones(len(grid))
    
#     pos_x = grid_in[:,0]
#     pos_y = grid_in[:,1]
#     pos_z = grid_in[:,2]

#     # Stack the calculated values along the third axis to create the grid
#     rays_in_values = np.stack((pos_x, pos_y, pos_z, -ori_x, -ori_y, -ori_z), axis=1)
#     rays_in_tensor = o3d.core.Tensor(rays_in_values, dtype=o3d.core.Dtype.Float32)

#     pos_x = grid_out[:,0]
#     pos_y = grid_out[:,1]
#     pos_z = grid_out[:,2]

#     rays_out_values = np.stack((pos_x, pos_y, pos_z, ori_x, ori_y, ori_z), axis=1)
#     rays_out_tensor = o3d.core.Tensor(rays_out_values, dtype=o3d.core.Dtype.Float32)       

#     ans_in = scene.cast_rays(rays_in_tensor)
#     ans_out = scene.cast_rays(rays_out_tensor)

#     hits_in = ans_in['t_hit'].numpy()
#     hits_out = ans_out['t_hit'].numpy()

#     opening_points = []
#     opening_colors = []
    
#     colors = np.zeros((len(hits_in), 3))
#     for i, c in enumerate(colors):
#         if hits_in[i] > 2*offset and hits_out[i] > 2*offset:
#             colors[i] = [0,1,0]
#             opening_points.append(grid[i])
#             opening_colors.append([0,1,0])

#         elif hits_in[i] <2*offset or hits_out[i] < 2*offset:
#             colors[i] = [0.5,0.5,0.5]
#             thickness = 2*offset-hits_out[i]-hits_in[i]
    
#             if 1.2*voxel_size < thickness < 0.5*wallThickness:
#                 # colors[i] = [1,0.5,0]
#                 opening_points.append(grid[i])
#                 opening_colors.append([0,1,0])


#     # grid_center_pcd.colors =o3d.utility.Vector3dVector(np.asarray(colors))
#     opening_pcd = o3d.geometry.PointCloud()
#     opening_pcd.points = o3d.utility.Vector3dVector(np.asarray(opening_points))
#     opening_pcd.colors = o3d.utility.Vector3dVector(np.asarray(opening_colors))
    
#     opening_pcd, ind = opening_pcd.remove_radius_outlier(nb_points = 100, radius = 0.1)

#     # o3d.visualization.draw_geometries([opening_pcd])

#     points = np.asarray(opening_pcd.points)
#     if len(points) > min_samples:

#         # Perform clustering using DBSCAN
#         dbscan = DBSCAN(eps=eps, min_samples=min_samples)
#         labels = dbscan.fit_predict(points)

#         # Extract unique cluster labels (excluding noise label -1)
#         unique_labels = np.unique(labels[labels != -1])

#         # Iterate over each cluster label and save corresponding points to a separate point cloud
#         for label in unique_labels:
#             cluster_points = points[labels == label]
#             cluster_pcd = o3d.geometry.PointCloud()
#             cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
#             openingpcds.append(cluster_pcd)
#     if show:
#         o3d.visualization.draw_geometries(openingpcds)
#     # o3d.visualization.draw_geometries([grid_center_pcd])
   
#     return openingpcds
    

In [22]:
# pcd_resolution = 0.01
# for n in wallBIMNodes:
    
#     length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
#     surface = length * n.height
#     n.openings1 = []

#     if not surface < 3 and n.height > 1.5 and length > 0.8:
                
#         n.openings1 = create_opening_pcds(startpoint = n.startpoint, endpoint = n.endpoint, height=n.height, resolution= 0.01, direction= n.normal, scene= scene, offset =0.5, path = None, show = False, min_samples = 1000, eps = 0.25, wallThickness = n.wallThickness, voxel_size=voxel_size)
        

In [23]:
# doors = []
# other = []
# for n in wallBIMNodes:
#     n.baseConstraint = next(l for l in levelNodes if l.subject.toPython() in [w.base_constraint for w in wallBIMNodes])
    
#     if len(n.openings1) > 0:
#         for opening in n.openings1:
#             # Extract the z-coordinates
#             points2 = np.asarray(opening.points)
#             z_values = points2[:, 2]
#             unique_z_values = np.unique(z_values)

#             #Compute the width of the opening
#             opening_width = 0.0
#             for z_value in unique_z_values:
#                 # Get points with the current z-value
#                 points_with_same_z = points2[z_values == z_value]

#                 # Find the outermost points based on XY coordinates
#                 min_x = np.min(points_with_same_z[:, 0])
#                 max_x = np.max(points_with_same_z[:, 0])
#                 min_y = np.min(points_with_same_z[:, 1])
#                 max_y = np.max(points_with_same_z[:, 1])

#                 # Compute the diagonal length of the bounding box
#                 diagonal_length = np.linalg.norm([max_x - min_x, max_y - min_y])

#                 if diagonal_length > opening_width:
#                     opening_width = diagonal_length
                    
#             print("Opening Width:", opening_width)
            
#             #Compute the Height of the door
#             lowest_z = np.min(z_values)
#             highest_z = np.max(z_values)
#             opening_height = highest_z - lowest_z   
#             print("Opening height:", opening_height)
            
#             #Depth should be equal to the wall thickness
            
#             if is_door(openingwidth = opening_width, openingheight = opening_height):
#                 doors.append(opening)
#                 print("This is most likely a door")

In [24]:
# # o3d.visualization.draw_geometries(doors)
# joined_pcd=gmu.join_geometries(doors)
# o3d.io.write_point_cloud(os.path.join(Path(os.getcwd()).parents[0]/'data', "doors.pcd"), joined_pcd)

## Method 2: Using the pointcloud and wall data to create an ortho foto of the wall and use object detection
(Can also be used to retrieve other elements)

In [25]:
def create_wall_ortho(startpoint, endpoint, height, resolution, direction, scene, offset =1, path = None, show = False, dominant = True, max_distance = 0.5, min_distance = 0.5):
    image_size = (int(np.sqrt(np.sum((endpoint - startpoint)**2)) / resolution)+1, int(height / resolution))
    
    min_z = np.min([endpoint[2], startpoint[2]])
    max_z = min_z + height
    num_z_steps = int(height /resolution)
    z_grid = np.linspace(min_z, max_z, num_z_steps)  # Adjust the number of grid points as needed
    z_grid = z_grid[::-1]
    xyz_grid = []
    for z in z_grid:
        start = startpoint.copy()
        end =  endpoint.copy()
        start[2] = z
        end[2] = z
        if not dominant:
            xyz_grid.append(points_on_line(start, end, resolution)[::-1])
        else: 
            xyz_grid.append(points_on_line(start, end, resolution))

    grid = np.asarray(xyz_grid).reshape((-1, 3), order='C') 
    ray_grid = grid + direction*offset
    
    #create rays for the in side (towards the dominant side
    ori_x = direction[0] * np.ones(len(ray_grid))
    ori_y = direction[1] * np.ones(len(ray_grid))
    ori_z = direction[2] * np.ones(len(ray_grid))
    
    pos_x = ray_grid[:,0]
    pos_y = ray_grid[:,1]
    pos_z = ray_grid[:,2]
    
    # Stack the calculated values along the third axis to create the grid
    rays_values = np.stack((pos_x, pos_y, pos_z, -ori_x, -ori_y, -ori_z), axis=1)
    rays_tensor = o3d.core.Tensor(rays_values, dtype=o3d.core.Dtype.Float32)
    
    ans= scene.cast_rays(rays_tensor) 
    
    
    triangle_ids = ans["primitive_ids"].numpy() # triangles     
    triangle_ids = triangle_ids.flatten()
    np.put(triangle_ids,np.where(triangle_ids==scene.INVALID_ID),triangle_colors.shape[0]-1) # replace invalid id's by last (which is the above added black color)
    
    # Get the hit distances for each ray
    hit_distances = ans["t_hit"].numpy().flatten()
    
    # Filter out hits that are too far or too close
    if max_distance is not None:
        triangle_ids[hit_distances > max_distance+offset] = triangle_colors.shape[0] - 1  # Set to black
        
    if min_distance is not None:
        triangle_ids[hit_distances < min_distance] = triangle_colors.shape[0] - 1  # Set to black
    
    colors = triangle_colors[triangle_ids]
    ortho = np.reshape(colors,(image_size[1],image_size[0],3))
    
    if show:
        plt.imshow(ortho)
        plt.show()
    if path:
        image = Image.fromarray((ortho * 255).astype(np.uint8))
        # Save the image
        image.save(path)
        
    return ortho

In [26]:
def fill_black_pixels(image:np.array,region:int=5)->np.array:
    """Fill in the black pixels in an RGB image given a search distance.\n
 
    Args:
        image (np.array)\n
        region (int, optional): search distance. Defaults to 5.\n
 
    Returns:
        np.array: image
    """
    kernel = np.ones((region,region),np.uint8)
    return cv2.morphologyEx(image, cv2.MORPH_CLOSE, kernel)

In [27]:
image_resolution = 0.01

for n in wallNodes:
    
    length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
    surface = length * n.height
    image_size = (int(length / image_resolution), int(n.height / image_resolution))
    n.orthos = []
    

    if not surface < 3 and n.height > 1.5 and length > 0.8:  
        #Create an ortho of the dominant side of the wall      
        ortho = create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = n.normal, scene=scene)
        ortho = fill_black_pixels(ortho, region = 10)
        n.orthos.append(ortho)
        #Also create an ortho of the other side of the wall
        if not n.wallThickness == 0.1: #Single faced wall only needs one side
            ortho = create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = -n.normal, scene=scene, dominant = False)
            ortho = fill_black_pixels(ortho, region = 10)
            n.orthos.append(ortho)

In [28]:
import torch
from PIL import Image
from torchvision.ops import box_convert

# Grounding DINO
from groundingdino.models import build_model
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
# from segment_anything import build_sam, SamPredictor 
import numpy as np


# diffusers
import torch

from huggingface_hub import hf_hub_download

In [29]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file) 
    model = build_model(args)
    args.device = device

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location=device)
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model   

In [30]:
# Use this command for evaluate the Grounding DINO model
# Or you can download the model by yourself
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

In [36]:
from torch.cuda.amp import autocast
with autocast():
    groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device="cuda:1")

final text_encoder_type: bert-base-uncased
Model loaded from /home/sdegeyter/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [37]:
count = 0
pointcloud = []


for n in wallNodes:
    TEXT_PROMPT = "Door"
    BOX_TRESHOLD = 0.30
    TEXT_TRESHOLD = 0.5
    
    n.boxes = []
    n.logits = []
    n.phrases = []
    
    if len(n.orthos) > 0:
        
        for ortho in n.orthos:
            boxes = None
            image = load_image(Image.fromarray((ortho * 255).astype(np.uint8)))

            boxes, logits, phrases = predict(
                model=groundingdino_model, 
                image=image, 
                caption=TEXT_PROMPT, 
                box_threshold=BOX_TRESHOLD, 
                text_threshold=TEXT_TRESHOLD
            )
            
            n.boxes.append(boxes)
            n.logits.append(logits)
            n.phrases.append(phrases)                  
                    

2024-05-16 14:54:35.259889: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 14:54:39.625484: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/python3.10/site-packages/cv2/../../lib64:
2024-05-16 14:54:39.625852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sdegeyter/.conda/envs/pointcept/lib/pyth

In [116]:
def create_point_cloud(img, startpoint, endpoint, box):
    # Define the UV coordinate ranges
    u_range = np.linspace(0, 1, img.shape[1])  # Width of the image corresponds to U
    v_range = np.linspace(0, 1, img.shape[0])  # Height of the image corresponds to V
    print(u_range)
    print(v_range)
    if not img[0].shape == img[1].shape:
        print("PROBLEM")
    # Initialize arrays to store points and colors
    points = []
    colors = []

    # Precalculate box boundaries
    box_boundaries_0 = []
    opening_width = int(np.asarray(box)[2]*img.shape[1])* image_resolution
    opening_height =   int(np.asarray(box)[3]*img.shape[0]) * image_resolution
    
    detection_center_u = int(np.asarray(box)[0]*img.shape[1]) * image_resolution
    detection_center_v = int(np.asarray(box)[1]*img.shape[0]) * image_resolution  
                
    u_min = detection_center_u - opening_width / 2
    u_max = detection_center_u + opening_width / 2
    v_min = detection_center_v - opening_height / 2
    v_max = detection_center_v + opening_height / 2
    
    box_boundaries_0.append((u_min, u_max, v_min, v_max))
            
    # Iterate over each pixel in the image
    if len(box_boundaries_0) > 0:
        for v, u, a in np.ndindex(img.shape):
            # Map UV coordinates to XYZ coordinates
            x = (1 - u_range[u]) * startpoint[0] + u_range[u] * endpoint[0]
            y = (1 - u_range[u]) * startpoint[1] + u_range[u] * endpoint[1]
            z = (1 - v_range[v]) * endpoint[2]  # Since V represents Z
            
            # Check if the point falls inside any of the boxes
            in_box_0 = False
            for box_boundary in box_boundaries_0:
                    u_min, u_max, v_min, v_max = box_boundary
                    if u_min <= u*image_resolution < u_max and v_min <= v*image_resolution < v_max:
                        in_box_0 = True
                        break
        
            # Assign the color based on whether the point is inside any of the boxes
            if in_box_0: 
                colors.append([0, 0, 1])  # Red
                points.append([x, y, z])

    return np.array(points), np.array(colors)

In [39]:
def extract_box_with_margin(image, box, margin = 10):
    # Extract center and size from the detection array
    width = int(np.asarray(box)[2]*image.shape[1])
    height =   int(np.asarray(box)[3]*image.shape[0])
    
    center_x = int(np.asarray(box)[0]*image.shape[1])
    center_y = int(np.asarray(box)[1]*image.shape[0])  
    

    # Calculate coordinates of the bounding box with extra margin
    x1 = int(center_x - width/2) - margin
    y1 = int(center_y - height/2) - margin
    x2 = int(center_x + width/2) + margin
    y2 = int(center_y + height/2) + margin

    # Ensure the box is within the image bounds
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    x2 = min(x2, image.shape[1])
    y2 = min(y2, image.shape[0])

    # Create a new image containing pixels within the bounding box
    extracted_image = np.copy(image[y1:y2, x1:x2])
    normalized_image = cv2.normalize(extracted_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return normalized_image.astype(np.uint8)

def compute_center(point1, point2):
    center = [(point1[0] + point2[0]) / 2,
              (point1[1] + point2[1]) / 2,
              (point1[2] + point2[2]) / 2]
    return center

In [145]:
def is_door(probability, width, height, reference_level, prob_weight=0.25, width_weight=0.25, height_weight=0.25, ref_level_weight=0.25):
    # Ensure parameters are within valid ranges
    if height < 1.5 or height > 2.5:
        return 0  # Invalid height for a door

    # Width score calculation with two ideal ranges and additional penalty
    if 0.6 <= width <= 1.0 or 1.2 <= width <= 2.0:
        width_score = 1
    elif width < 0.6:
        width_score = width / 0.6 - 0.2
    elif width < 1.2:
        width_score = max(0, (1.2 - width) / (1.2 - 1.0)) - 0.2
    elif width > 2.0:
        width_score = max(0, 2.0 / width) - 0.5
    else:
        width_score = 0 - 0.2  # Should not reach here

    # Height score calculation with ideal range of 2m to 2.4m and additional penalty
    if 2.0 <= height <= 2.4:
        height_score = 1
    elif height < 2.0:
        height_score = max(0, (height - 1.5) / 0.5) - 0.2
    else:
        height_score = max(0, (2.5 - height) / 0.1) - 0.2

    # Reference level score calculation with additional penalty
    if reference_level <= 0.05:
        reference_level_score = 1
    else:
        reference_level_score = max(0, 1 - (reference_level - 0.05) / 0.95) - 0.2

    # Return 0 if any score is 0
    if width_score <= 0 or height_score <= 0 or reference_level_score <= 0:
        return 0

    # Normalize weights so that their sum is 1
    total_weight = prob_weight + width_weight + height_weight + ref_level_weight
    normalized_prob_weight = prob_weight / total_weight
    normalized_width_weight = width_weight / total_weight
    normalized_height_weight = height_weight / total_weight
    normalized_ref_level_weight = ref_level_weight / total_weight

    # Weighted scores
    weighted_prob = probability * normalized_prob_weight
    weighted_width = width_score * normalized_width_weight
    weighted_height = height_score * normalized_height_weight
    weighted_ref_level = reference_level_score * normalized_ref_level_weight

    # Combine weighted scores
    combined_score = weighted_prob + weighted_width + weighted_height + weighted_ref_level


        
    # print("Probability: %s - SCORE: %s" %(probability, weighted_prob))
    # print("Width: %s - SCORE: %s" %(width, weighted_width))
    # print("Height: %s - SCORE: %s" %(height, weighted_height))
    # print("Reference Level: %s - SCORE: %s" %(reference_level, weighted_ref_level))
    # print("==========================================================================")
    # print("Total SCORE: %s" %(combined_score))
    # print("")

    return combined_score

In [98]:
def box_to_corners(box):
    u_center, v_center, u_size, v_size = box
    half_u_size = u_size / 2
    half_v_size = v_size / 2
    u_min = u_center - half_u_size
    u_max = u_center + half_u_size
    v_min = v_center - half_v_size
    v_max = v_center + half_v_size
    return u_min, v_min, u_max, v_max

def compute_iou(box1, box2):
    # Convert boxes to corners
    u_min1, v_min1, u_max1, v_max1 = box_to_corners(box1)
    u_min2, v_min2, u_max2, v_max2 = box_to_corners(box2)
    
    # Calculate intersection coordinates
    inter_u_min = max(u_min1, u_min2)
    inter_v_min = max(v_min1, v_min2)
    inter_u_max = min(u_max1, u_max2)
    inter_v_max = min(v_max1, v_max2)
    
    # Calculate intersection area
    inter_area = max(0, inter_u_max - inter_u_min) * max(0, inter_v_max - inter_v_min)
    
    # Calculate areas of each box
    area1 = (u_max1 - u_min1) * (v_max1 - v_min1)
    area2 = (u_max2 - u_min2) * (v_max2 - v_min2)
    
    # Calculate union area
    union_area = area1 + area2 - inter_area
    
    # Calculate IoU
    iou = inter_area / union_area if union_area > 0 else 0
    
    return iou
def compute_parameter_similarity(params1, params2):
    # Example: use Euclidean distance and convert it to a similarity score
    distance = np.linalg.norm(np.array(params1) - np.array(params2))
    similarity = 1 / (1 + distance)  # similarity decreases with distance
    return similarity

def compute_doorness_similarity(score1, score2):
    # Assuming score1 and score2 are probabilities or confidence scores
    # Convert them into numpy arrays for easier computation
    score1 = np.array(score1)
    score2 = np.array(score2)
    
    # Compute Euclidean distance between the scores
    distance = np.linalg.norm(score1 - score2)
    
    # Similarity is inversely proportional to distance
    similarity = 1 / (1 + distance)
    
    return similarity

def find_best_matches(boxes1, boxes2, params1, params2, iou_weight=0.33, param_weight=0.33, doorness_weight=0.33):
    matches = []
    used_boxes2 = set()

    for i, box1 in enumerate(boxes1):
        max_score = -1
        best_match = -1

        for j, box2 in enumerate(boxes2):
            if j in used_boxes2:
                continue
            
            iou = compute_iou(box1, box2)
            param_similarity = compute_parameter_similarity(params1[i][0:2], params2[j][0:2])
            doorness_similarity = compute_doorness_similarity(params1[i][4:5], params2[j][4:5])
            
            # Combine IoU, parameter similarity, and doorness score into a single score
            combined_score = iou_weight * iou + param_weight * param_similarity + doorness_weight * doorness_similarity
            
            if combined_score > max_score:
                max_score = combined_score
                best_match = j
        
        if best_match != -1:
            matches.append((i, best_match, max_score))
            used_boxes2.add(best_match)
        else:
            matches.append((i, None, None))

    return matches


In [56]:
def combine_boxes(box1, box2):
    u_min1, v_min1, u_max1, v_max1 = box_to_corners(box1)
    u_min2, v_min2, u_max2, v_max2 = box_to_corners(box2)

    combined_u_min = min(u_min1, u_min2)
    combined_v_min = min(v_min1, v_min2)
    combined_u_max = max(u_max1, u_max2)
    combined_v_max = max(v_max1, v_max2)

    combined_u_center = (combined_u_min + combined_u_max) / 2
    combined_v_center = (combined_v_min + combined_v_max) / 2
    combined_u_size = combined_u_max - combined_u_min
    combined_v_size = combined_v_max - combined_v_min

    return np.array([[combined_u_center, combined_v_center, combined_u_size, combined_v_size]])


In [158]:
def line_with_width_coordinates(start, end, percentage, width):
    # Calculate the coordinates of the point on the line segment
    point_on_line = start + (end - start) * percentage
    
    # Calculate the direction vector of the line segment
    direction = (end - start) / np.linalg.norm(end - start)
    
    # Calculate the offset for the new line
    offset = direction * (width / 2)
    
    # Calculate the start and end points of the new line
    new_start = point_on_line - offset
    new_end = point_on_line + offset
    
    return [new_start, new_end]

In [169]:
doorNodes = []
pointcloud = []

wall = 0
count = 0
for n in wallNodes: #^22:24 interessante muren
    potential_door_boxes_wall = []
    potential_door_info_wall = []
    potential_door_Nodes = []
    if len(n.orthos) > 0 and (len(n.boxes[0]) > 0 or len(n.boxes[1]) > 0):
        
        for j, boxes in enumerate(n.boxes):
            potential_door_boxes_face = []
            potential_door_info_face = []
            if len(boxes) > 0: 
                for i, box in enumerate(boxes):
                    probability = float(n.logits[j][i])
                    # print("Probability: ", probability)
                    
                    opening_width = round(int(np.asarray(box)[2]*n.orthos[j].shape[1])* image_resolution, 2)
                    # print("Opening Width:", opening_width)
                    
                    opening_height = round(int(np.asarray(box)[3]*n.orthos[j].shape[0]) * image_resolution, 2)
                    # print("Opening height:", opening_height)
                    
            
                    detection_center_u = int(np.asarray(box)[0]*n.orthos[j].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(box)[1]*n.orthos[j].shape[0]) * image_resolution
                    
                    reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                    # print("Reference Level:", reference_level)
                    
                    score = is_door(probability, opening_width, opening_height, reference_level)
                    # if stand == 1:
                    #     points, colors = create_point_cloud(n.orthos[stand], n.endpoint,n.startpoint, box)
                    # else:
                    #     points, colors = create_point_cloud(n.orthos[stand],n.startpoint, n.endpoint, box)
                    if score >= 0.5:
                        box1 = copy.deepcopy(box)
                        box = box.unsqueeze(0)

                        if j == 1:
                            box1[0] = 1-box1[0]

                        image = extract_box_with_margin(n.orthos[j], box[0])
                        image = image[...,::-1] # BGR to RGB
                        # if wall == 53:
                        #     print(wall)
                        #     Image.fromarray(image)#.save(os.path.join(output_folder,(str(wall) +"-"+ str(stand)+"-" + str(count) +'-DOOR.png')))
                        #     annotated_frame_doorness = annotate(image_source= n.orthos[j], boxes=box, logits=torch.from_numpy(np.array([score])), phrases=[n.phrases[j][i]])
                        #     # annotated_frame = annotate(image_source= n.orthos[j], boxes=box, logits=n.logits[j][i].unsqueeze(0), phrases=[n.phrases[j][i]])
                        #     annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        #     Image.fromarray(annotated_frame_doorness).save(os.path.join(output_folder,(str(wall) +"-"+ str(j)+"-" + str(count) + '-DOOR-'+str(score*100)+'.png')))
                        #     print("%s : Door => Referencelevel: %s; Width: %s; Height: %s" %((str(wall) + "-" + str(j) + "-" + str(count)), reference_level, opening_width, opening_height))

                        # count += 1
                        potential_door_boxes_face.append(np.asarray(box1))
                        potential_door_info_face.append([opening_width, opening_height, image, reference_level, score])
                        
                    # else:
                    #     image = extract_box_with_margin(n.orthos[stand], box.unsqueeze(0)[0])
                    #     image = image[...,::-1] # BGR to RGB
                    #     Image.fromarray(image)#.save(os.path.join(output_folder,(str(wall) +"-"+ str(stand)+"-" + str(count) +'-DOOR.png')))
                    #     annotated_frame = annotate(image_source= n.orthos[stand], boxes=box.unsqueeze(0), logits=n.logits[stand][i].unsqueeze(0), phrases=[n.phrases[stand][i]])
                    #     annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    #     Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(wall) +"-"+ str(stand)+"-" + str(count) +'-DOOR-'+str(score*100)+'.png')))
                    #     print("%s : No Door => Referencelevel: %s; Width: %s; Height: %s" %((str(wall) + "-" + str(stand) + "-" + str(count)), reference_level, opening_width, opening_height))
                    #     count += 1
                        
                    #     if points.size > 0:
                    #         # Create Open3D point cloud
                    #         pcd_o3d = o3d.geometry.PointCloud()
                    #         pcd_o3d.points = o3d.utility.Vector3dVector(points)
                    #         pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
                    #         o3d.io.write_point_cloud(os.path.join(output_folder,(str(wall) + "-" + str(stand) + "-" + str(count) + '-DOOR.pcd')), pcd_o3d)
                    #         base_points = points[points[:, 2] == 0, :2]
                    #         # print(base_points)
                    #         min_x_index = np.argmin(points[:, 0])
                    #         max_x_index = np.argmax(points[:, 0])
                                                        
                    #         startpoint = [points[min_x_index][0],points[min_x_index][1],0 ]
                    #         endpoint = [points[max_x_index][0],points[max_x_index][1],0 ]
                    #         center = compute_center(startpoint, endpoint)
                    #         distance = np.sqrt((endpoint[0] - startpoint[0])**2 + (endpoint[1] - startpoint[1])**2)
                            
                            
                    #         # pcd_node = PointCloudNode(resource = pcd_o3d, path = os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) + "-" + str(stand) + "-" + str(count) + '-DOOR.pcd')))
                            
                    #     on = Node()
                    #     on.width = opening_width
                    #     on.height = opening_height
                    #     on.depth = n.wallThickness
                    #     on.pcd = pcd_o3d
                    #     on.image = image
                    #     on.startpoint = startpoint
                    #     on.endpoint = endpoint
                    #     on.center = center
                    #     on.normal = n.normal
                    #     on.host = n
                    #     potential_doors.append(on)            
            potential_door_boxes_wall.append(potential_door_boxes_face)
            potential_door_info_wall.append(potential_door_info_face)
        
        if len(potential_door_boxes_wall) == len(n.orthos) and not len(potential_door_boxes_wall[0]) == 0 and not len(potential_door_boxes_wall[1]) == 0:
            matches = find_best_matches(potential_door_boxes_wall[0], potential_door_boxes_wall[1], potential_door_info_wall[0], potential_door_info_wall[1])
            for id0, id1, bestscore in matches:
                if not id0 == None and not id1 == None:
                    # compare the parameters of the different matches
                    info0 = potential_door_info_wall[0][id0]
                    info1 = potential_door_info_wall[1][id1]
                    image = info0[2]
                    # boxestemp = np.array([potential_door_boxes_wall[0][id0],potential_door_boxes_wall[1][id1]])
                    detectionbox = combine_boxes(potential_door_boxes_wall[0][id0], potential_door_boxes_wall[1][id1])
                    # Image.fromarray(image).save(os.path.join(output_folder,(str(count) +'-DOOR.png')))
                    # annotated_frame = annotate(image_source= n.orthos[0], boxes=torch.from_numpy(detectionbox), logits=torch.from_numpy(np.array([bestscore])), phrases=[n.phrases[0][id0]])
                    # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                    # Image.fromarray(annotated_frame).save(os.path.join(output_folder,(str(count) +'-DOOR.png')))
                    
                    
                    opening_width = round(int(np.asarray(detectionbox)[0][2]*n.orthos[j].shape[1])* image_resolution, 2)
                    # print("Opening Width:", opening_width)
                    
                    opening_height = round(int(np.asarray(detectionbox)[0][3]*n.orthos[j].shape[0]) * image_resolution, 2)
                    
                    detection_center_u = int(np.asarray(detectionbox)[0][0]*n.orthos[j].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(detectionbox)[0][1]*n.orthos[j].shape[0]) * image_resolution
                    reference_level = round((n.orthos[j].shape[0]*image_resolution) - (detection_center_v + opening_height/2), 2)
                    
                    # print("%s : Door Score: %s => Referencelevel: %s; Width: %s; Height: %s" %((str(count)), bestscore, reference_level, opening_width, opening_height))
                    count += 1
                    boundaryPoints = line_with_width_coordinates(n.startpoint, n.endpoint, detectionbox[0][0], opening_width)
                    
                    doornode = Node()
                    doornode.name= "Doors_" + str(count)
                    doornode.axis=o3d.geometry.LineSet(points=o3d.utility.Vector3dVector(boundaryPoints),lines=o3d.utility.Vector2iVector([[0,1]])).paint_uniform_color([0,0,1])
                    doornode.startPoint= boundaryPoints[0]
                    doornode.endPoint= boundaryPoints[1]
                    doornode.doorWidth = np.round(np.linalg.norm(boundaryPoints[0] - boundaryPoints[1]),2)
                    doornode.height = opening_height
                    
                    pointList=[]
                    points=np.asarray(doornode.axis.points)
                    # pointList.extend(points+n.sign*n.normal*n.wallThickness/2)
                    pointList.extend(points+n.normal*n.wallThickness/2)

                    # pointList.extend(points-n.sign*n.normal*n.wallThickness/2)
                    pointList.extend(points-n.normal*n.wallThickness/2)

                    pointList.extend(np.array(pointList)+np.array([0,0,doornode.height]))
                    pcd=o3d.geometry.PointCloud(points=o3d.utility.Vector3dVector(pointList))

                    box=pcd.get_oriented_bounding_box()
                    doornode.door=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(box)
                    doornode.door.paint_uniform_color(ut.literal_to_array(n.color))
                    doornode.doorBox=o3d.geometry.LineSet.create_from_oriented_bounding_box(box)
                    doornode.doorBox.paint_uniform_color([0,0,1])
                    
                    doornode.host = n
                    doorNodes.append(doornode)


                    print(f'name: {doornode.name}, doorWidth: {doornode.doorWidth}, height: {doornode.height}')
                    
                    # on.width = opening_width
                    # on.height = opening_height
                    # on.depth = n.wallThickness
                    # # on.pcd = pcd_o3d
                    # on.image = image
                    # # on.startpoint = startpoint
                    # # on.endpoint = endpoint
                    # # on.center = center
                    # on.normal = n.normal
                    # on.host = n
                    # potential_doors.append(on)
                    # points, colors = create_point_cloud(n.orthos[0],n.startpoint, n.endpoint, detectionbox[0])
                    # if points.size > 0:
                    # #   # Create Open3D point cloud
                    #     pcd_o3d = o3d.geometry.PointCloud()
                    #     pcd_o3d.points = o3d.utility.Vector3dVector(points)
                    #     pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
                    #     o3d.io.write_point_cloud(os.path.join(output_folder,(str(count) + '-DOOR.pcd')), pcd_o3d)
                    #     base_points = points[points[:, 2] == 0, :2]
                    #     # print(base_points)
                    #     min_x_index = np.argmin(points[:, 0])
                    #     max_x_index = np.argmax(points[:, 0])
                                                    
                    #     startpoint = [points[min_x_index][0],points[min_x_index][1],0 ]
                    #     endpoint = [points[max_x_index][0],points[max_x_index][1],0 ]
                    #     center = compute_center(startpoint, endpoint)
                    #     distance = np.sqrt((endpoint[0] - startpoint[0])**2 + (endpoint[1] - startpoint[1])**2)
                        
                        # pcd_node = PointCloudNode(resource = pcd_o3d, path = os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) + "-" + str(stand) + "-" + str(count) + '-DOOR.pcd')))
                

        
        #compute the IoU of the detections on bopth sides
        #Find a matching box with a large IoU on both
        
            
        
        
         
        wall += 1

name: Doors_1, doorWidth: 1.1, height: 2.29
name: Doors_2, doorWidth: 1.11, height: 2.32
name: Doors_3, doorWidth: 1.12, height: 2.33
name: Doors_4, doorWidth: 1.11, height: 2.31
name: Doors_5, doorWidth: 1.58, height: 2.34
name: Doors_6, doorWidth: 1.59, height: 2.32
name: Doors_7, doorWidth: 1.6, height: 2.3
name: Doors_8, doorWidth: 1.62, height: 2.28
name: Doors_9, doorWidth: 3.36, height: 2.3
name: Doors_10, doorWidth: 1.12, height: 2.32
name: Doors_11, doorWidth: 2.2, height: 2.33
name: Doors_12, doorWidth: 1.57, height: 2.29
name: Doors_13, doorWidth: 1.61, height: 2.28
name: Doors_14, doorWidth: 1.59, height: 2.32
name: Doors_15, doorWidth: 1.61, height: 2.3
name: Doors_16, doorWidth: 0.87, height: 2.12
name: Doors_17, doorWidth: 0.95, height: 2.14
name: Doors_18, doorWidth: 0.87, height: 2.12
name: Doors_19, doorWidth: 0.89, height: 2.1
name: Doors_20, doorWidth: 0.94, height: 2.14
name: Doors_21, doorWidth: 0.9, height: 2.11
name: Doors_22, doorWidth: 0.96, height: 2.15
name:

In [170]:
def write_obj_with_submeshes(filename, meshes, mesh_names):
    """
    Write multiple Open3D TriangleMesh objects to a single OBJ file with submeshes.

    Parameters:
    - filename: str, the name of the output OBJ file.
    - meshes: list of open3d.geometry.TriangleMesh, the meshes to write.
    - mesh_names: list of str, the names of the submeshes.
    """
    if len(meshes) != len(mesh_names):
        raise ValueError("meshes and mesh_names must have the same length")

    vertex_offset = 1  # OBJ files are 1-indexed
    with open(filename, 'w') as file:
        for mesh, name in zip(meshes, mesh_names):
            file.write(f"g {name}\n")  # Start a new group for the submesh

            # Write vertices
            for vertex in mesh.vertices:
                file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

            # Write faces, adjusting indices based on the current offset
            for triangle in mesh.triangles:
                adjusted_triangle = triangle + vertex_offset
                file.write(f"f {adjusted_triangle[0]} {adjusted_triangle[1]} {adjusted_triangle[2]}\n")

            # Update the vertex offset for the next mesh
            vertex_offset += len(mesh.vertices)

# Example usage:
# Assuming mesh1 and mesh2 are your Open3D TriangleMesh objects
mesh1 = o3d.geometry.TriangleMesh.create_sphere(radius=1.0)
mesh1.compute_vertex_normals()

mesh2 = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
mesh2.compute_vertex_normals()

write_obj_with_submeshes(os.path.join(output_folder,f'{ut.get_filename(f)}_doors.obj') , [n.door for n in doorNodes], [n.name for n in doorNodes])
    

In [ ]:
# def plot_lines(lines, show_centers=True):
#     fig, ax = plt.subplots()

#     # Iterate over each line and plot it with a different color
#     if len(lines[0]) == 2:
#         for i, (start, end) in enumerate(lines):
#             color = plt.cm.viridis(i / len(lines))  # Get a color from the Viridis colormap
#             ax.plot([start[0], end[0]], [start[1], end[1]], color=color)
            
#             if show_centers:
#                 # Compute the center of the line
#                 center = compute_center(start, end)
                
#                 # Plot the center with the same color as the line
#                 ax.scatter(center[0], center[1], color=color, label=f'Center {i+1}')

#         ax.set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
#         ax.set_xlabel('X')
#         ax.set_ylabel('Y')
#         ax.set_title('Lines with Centers')
#         if show_centers:
#             ax.legend()
#         plt.show()
#     else:
#         for i, (start, end, _) in enumerate(lines):
#             color = plt.cm.viridis(i / len(lines))  # Get a color from the Viridis colormap
#             ax.plot([start[0], end[0]], [start[1], end[1]], color=color)
            
#             if show_centers:
#                 # Compute the center of the line
#                 center = compute_center(start, end)
                
#                 # Plot the center with the same color as the line
#                 ax.scatter(center[0], center[1], color=color, label=f'Center {i+1}')

#         ax.set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
#         ax.set_xlabel('X')
#         ax.set_ylabel('Y')
#         ax.set_title('Lines with Centers')
#         if show_centers:
#             ax.legend()
#         plt.show()

In [ ]:
# def compute_normal(start, end):
#     """
#     Compute the normal vector of a line given its start and end points.
    
#     Parameters:
#         start (tuple): The start point of the line.
#         end (tuple): The end point of the line.
    
#     Returns:
#         numpy.ndarray: The computed normal vector.
#     """
#     # Compute the direction vector of the line
#     direction = np.array(end) - np.array(start)
#     # Compute the normal vector by rotating the direction vector 90 degrees
#     normal = np.array([-direction[1], direction[0]])
#     # Normalize the normal vector
#     normal /= np.linalg.norm(normal)
    
#     return normal

# # Function to calculate the center point of a line segment
# def calculate_center(line):
#     startpoint, endpoint, _ = line
#     return [(startpoint[0] + endpoint[0]) / 2, (startpoint[1] + endpoint[1]) / 2]

# # Function to calculate the distance between two points
# def calculate_distance(point1, point2):
#     return np.linalg.norm(np.array(point1) - np.array(point2))

In [ ]:
# # Function to cluster lines based on normals and centers
# def cluster_lines(lines, distance_threshold=1, equal_normals = True):
    
#     cluster_ids = [-1] * len(lines)
#     current_cluster_id = 0
#     if equal_normals:
#         normals = set(tuple(line[2]) for line in lines)  # Convert numpy array to tuple
#         for normal in normals:
#             lines_with_normal = [line for line in lines if np.array_equal(line[2], normal)]
            
#             centers = [calculate_center(line) for line in lines_with_normal]
            
#             # Using DBSCAN for clustering centers
#             dbscan = DBSCAN(eps=distance_threshold, min_samples=1, metric=calculate_distance)
#             labels = dbscan.fit_predict(centers)
            
#             unique_labels = set(labels)
#             for label in unique_labels:
#                 cluster_lines = [lines_with_normal[i] for i in range(len(lines_with_normal)) if labels[i] == label]
#                 for line in cluster_lines:
#                     line_index = None
#                     for i, l in enumerate(lines):
#                         if np.array_equal(l, line):
#                             line_index = i
#                             break
#                     cluster_ids[line_index] = current_cluster_id
#                 current_cluster_id += 1
#     else:            
#         centers = [calculate_center(line) for line in lines]
        
#         # Using DBSCAN for clustering centers
#         dbscan = DBSCAN(eps=distance_threshold, min_samples=1, metric=calculate_distance)
#         labels = dbscan.fit_predict(centers)
        
#         unique_labels = set(labels)
#         for label in unique_labels:
#             cluster_lines = [lines[i] for i in range(len(lines)) if labels[i] == label]
#             for line in cluster_lines:
#                 line_index = None
#                 for i, l in enumerate(lines):
#                     if np.array_equal(l, line):
#                         line_index = i
#                         break
#                 cluster_ids[line_index] = current_cluster_id
#             current_cluster_id += 1
    
#     return cluster_ids

Eerste clustering om detecties langst beide kanten te bundelen

In [ ]:
# def find_outer_points(points):
#     # Convert points to numpy array for easier manipulation
#     points = np.array(points)
    
#     # Find the index of the point with the smallest coordinate along the line
#     min_index = np.argmin(points[:, 0])  # Assuming the line is along the x-axis
    
#     # Find the index of the point with the largest coordinate along the line
#     max_index = np.argmax(points[:, 0])  # Assuming the line is along the x-axis
    
#     # Retrieve the two outer points
#     outer_point_min = points[min_index]
#     outer_point_max = points[max_index]
    
#     return outer_point_min, outer_point_max

In [ ]:
# def retrieve_all_clusters_indices(cluster_labels):
#     all_clusters_indices = []
    
#     # Get unique cluster labels
#     unique_labels = np.unique(cluster_labels)
    
#     # Iterate through each unique cluster label
#     for cluster_label in unique_labels:
#         if not cluster_label == -1:
#             cluster_indices = [i for i, label in enumerate(cluster_labels) if label == cluster_label]
#             all_clusters_indices.append(cluster_indices)
#         else: 
#             for i, label in enumerate(cluster_labels):
#                 if label == cluster_label:
#                     all_clusters_indices.append([i])
    
#     return all_clusters_indices

# def are_normals_same_direction(objects):
#     # Check if all normals have the same direction
#     reference_normal = objects[0].normal
#     for obj in objects[1:]:
#         if not np.allclose(obj.normal, reference_normal):
#             return False
#     return True

# def merge_objects(openings):
#     if not are_normals_same_direction(openings):
#         on = openings[0]
#         return on
    
#     # Merge objects by finding extreme start and end points
#     on = Node()
#     on.startpoint, on.endpoint = find_outer_points([obj.startpoint for obj in openings] + [obj.endpoint for obj in openings])
#     on.height = sum(n.height for n in openings)/len(openings)
#     on.depth = openings[0].depth
#     on.pcd = [n.pcd for n in openings]
#     on.image = [n.image for n in openings]
#     on.center = compute_center(on.startpoint, on.endpoint)
#     on.normal = openings[0].normal
#     on.host = openings[0].host
#     on.width = np.sqrt((on.endpoint[0] - on.startpoint[0])**2 + (on.endpoint[1] - on.startpoint[1])**2)
    
#     return on

In [ ]:
# lines = []

# for n in potential_doors:
#     lines.append([n.startpoint, n.endpoint, n.normal])
# print(len(lines))
# # plot_lines(lines)
# clustered_ids = cluster_lines(lines) 
# # plot_clustered_lines_separately(lines, clustered_ids)
# all_clusters_indices = retrieve_all_clusters_indices(clustered_ids)
# clustered_lines = []

# for cluster_indices in all_clusters_indices:
#     openings = []
#     for i in cluster_indices:
#         openings.append(potential_doors[i])
#     on = merge_objects(openings)
#     clustered_lines.append(on)
    
# lines = []
# for n in clustered_lines:
#     lines.append([n.startpoint, n.endpoint])
    
# print(len(lines))

# plot_lines(lines, show_centers=False)

In [ ]:
# def calculate_percentage_black_pixels(image):
#     # Check if the image is a color image (3 channels) or a grayscale image (1 channel)
#     if len(image.shape) == 3 and image.shape[2] == 3:
#         # Convert the color image to grayscale
#         grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     elif len(image.shape) == 2:
#         # Image is already grayscale
#         grayscale_image = image
#     else:
#         # Handle other cases or raise an error
#         raise ValueError("Unsupported image format. Expected color (3 channels) or grayscale (1 channel).")

#     # # Convert the image to grayscale
#     # grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Count black pixels
#     black_pixel_count = cv2.countNonZero(grayscale_image)

#     # Calculate total number of pixels
#     total_pixels = grayscale_image.shape[0] * grayscale_image.shape[1]

#     # Calculate percentage of black pixels
#     percentage_black_pixels = (black_pixel_count / total_pixels) * 100

#     return percentage_black_pixels

In [ ]:
# lines = []

# for n in clustered_lines:
#     lines.append([n.startpoint, n.endpoint, n.normal])
    
# clustered_ids = cluster_lines(lines, distance_threshold=1.2, equal_normals = False)
# all_clusters_indices = retrieve_all_clusters_indices(clustered_ids)

# openings = []
# for cluster in all_clusters_indices:
#     if len(cluster) > 1:
#         l = [clustered_lines[i] for i in cluster]
        
#         # Plotting the images
#         # fig, axes = plt.subplots(1, len(l))            
        
#         lines = []
#         i = 0
#         for j, n in enumerate(l):
#             lines.append([n.startpoint, n.endpoint, n.normal])
#             # axes[i].imshow(Image.fromarray(n.image[0][0]))
#             # axes[i].axis('off')
#             i+=1
#             if calculate_percentage_black_pixels(n.image[0][0]) < 50:
#                 openings.append(n)
#                 for jj, n2 in enumerate(l): 
#                     if not jj == j:
#                         if np.array_equal(n.normal, n2.normal):
#                             openings.append(n2)
                
#         # plot_lines(lines)
#     else:
#         openings.append(clustered_lines[cluster[0]])

In [ ]:
# lines =[]
# for n in openings:
#     # n.center = compute_center(n.startpoint, n.endpoint)
#     lines.append([n.startpoint, n.endpoint, n.normal])
    
    
# print(len(lines))
# plot_lines(lines, show_centers=False)

In [ ]:
# # Compare ground truth nodes with computed nodes
# # print(len(groundtruthNodes))
# for ground_truth_node in groundtruthNodes:
#     min_distance = float('inf')
#     best_node = None
#     for computed_node in openings:
#         # print(ground_truth_node.center)
#         # print(compute_center(computed_node.startpoint, computed_node.endpoint))
#         # print(computed_node.center)
#         distance = calculate_distance(ground_truth_node.center, computed_node.center)
#         if distance < min_distance:
#             min_distance = distance
#             best_node = computed_node
#     print('Found match on: %s'% (min_distance))
#     print('WIDTH = GT: %s, Computed: %s' %(ground_truth_node.width, best_node.width))
#     print('DEPTH = GT: %s, Computed: %s' %(ground_truth_node.depth, best_node.depth))
#     print('HEIGHT = GT: %s, Computed: %s' %(ground_truth_node.height, best_node.height))

In [ ]:
# import matplotlib.pyplot as plt

# # Lists to store coordinates of matched, unmatched ground truth nodes, and unmatched computed nodes
# matched_gt_x = []
# matched_gt_y = []
# matched_near_gt_x = []
# matched_near_gt_y = []
# unmatched_gt_x = []
# unmatched_gt_y = []
# unmatched_calc_x = []
# unmatched_calc_y = []

# # Iterate through each ground truth node
# for ground_truth_node in groundtruthNodes:
#     min_distance = float('inf')
#     best_node = None
    
#     # Iterate through each computed node
#     for computed_node in openings:
#         distance = calculate_distance(ground_truth_node.center, computed_node.center)
        
#         if distance < min_distance:
#             min_distance = distance
#             best_node = computed_node
    
#     # Check if a match was found
#     if min_distance < 0.25:
#         matched_gt_x.append(ground_truth_node.center[0])
#         matched_gt_y.append(ground_truth_node.center[1])
#     elif min_distance < 0.5:  # Check if the matched node is less than 1m
#         matched_near_gt_x.append(ground_truth_node.center[0])
#         matched_near_gt_y.append(ground_truth_node.center[1])
#     else:
#         unmatched_gt_x.append(ground_truth_node.center[0])
#         unmatched_gt_y.append(ground_truth_node.center[1])

# # Iterate through each computed node and check if it was matched
# for computed_node in openings:
#     matched = False
#     for ground_truth_node in groundtruthNodes:
#         distance = calculate_distance(ground_truth_node.center, computed_node.center)
#         if distance < 0.5:
#             matched = True
#             break
#     if not matched:
#         unmatched_calc_x.append(computed_node.center[0])
#         unmatched_calc_y.append(computed_node.center[1])

# # Plot matched ground truth nodes in green
# plt.scatter(matched_gt_x, matched_gt_y, color='green', label='Matched Ground Truth (<0.5m)')

# # Plot matched ground truth nodes in orange (between 0.5m and 1m)
# plt.scatter(matched_near_gt_x, matched_near_gt_y, color='orange', label='Matched Ground Truth (0.5m-1m)')

# # Plot unmatched ground truth nodes in gray
# plt.scatter(unmatched_gt_x, unmatched_gt_y, color='gray', label='Unmatched Ground Truth')

# # Plot unmatched computed nodes in red
# plt.scatter(unmatched_calc_x, unmatched_calc_y, color='red', label='Unmatched Computed')

# plt.xlabel('X Coordinate')
# plt.ylabel('Y Coordinate')
# plt.title('Matching Results')
# plt.legend()
# plt.grid(True)
# plt.show()
